In [20]:
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


In [48]:
# Metric results for XGB on each benchamrked scoring interval 
# 15 fold CV giving 15 model performances per each socring interval
rsquared = [
    [0.45634346, 0.45634309, 0.46720414, 0.53580059, 0.57091602,
       0.46280731, 0.4935618 , 0.53677778, 0.382604  , 0.56331327,
       0.51347667, 0.51085961, 0.48929966, 0.536598  , 0.45951789], #0.5
    [0.56464358, 0.47999782, 0.56571044, 0.57966928, 0.62554679,
       0.51259351, 0.53375816, 0.62961087, 0.50366574, 0.63775923,
       0.56679491, 0.54655718, 0.57584658, 0.58425077, 0.46782443], #0.55
    [0.64646241, 0.53280181, 0.61769064, 0.65956511, 0.67255865,
       0.57570432, 0.60962886, 0.65997764, 0.5450545 , 0.69957268,
       0.65669438, 0.61781818, 0.62690541, 0.64047937, 0.50433829], #0.6
    [0.68777572, 0.56812135, 0.65901243, 0.68438677, 0.71190161,
       0.60903237, 0.67146768, 0.68490953, 0.57545748, 0.8133305,
       0.67938775, 0.64292705, 0.73683909, 0.64411455, 0.57301455], #0.65
    [0.72481275, 0.61634826, 0.68385177, 0.7341573 , 0.72475438,
       0.62767679, 0.70360632, 0.77061093, 0.64673306, 0.80856483,
       0.69638464, 0.66650218, 0.77264233, 0.70101662, 0.57129934], #0.7
    [0.7391932 , 0.59699563, 0.77811019, 0.77034099, 0.73011032,
       0.65258079, 0.8767709, 0.78212505, 0.65524694, 0.79146323,
       0.8202244, 0.6927761 , 0.83066508, 0.69552579, 0.5458303], #0.75
    [0.74034652, 0.64788809, 0.72896998, 0.77646792, 0.71746258,
       0.67124173, 0.7561366 , 0.78296082, 0.65660295, 0.881005  ,
       0.75959853, 0.71971786, 0.80844378, 0.73828392, 0.60823481] #0.8
]


mse = [
    [-0.04244925, -0.05463119, -0.05009597, -0.04321857, -0.03660862,
       -0.04670382, -0.03503022, -0.03695132, -0.06444117, -0.05112117,
       -0.05048046, -0.04733181, -0.03709436, -0.04430145, -0.04983393], #0.5
    [-0.03606417, -0.05172759, -0.04219071, -0.0393957 , -0.03401385,
       -0.04462899, -0.03450002, -0.03089786, -0.05186723, -0.04265159,
       -0.04546227, -0.04447348, -0.03247131, -0.0410435 , -0.05112777], #0.55
    [-0.03140706, -0.04658561, -0.03881302, -0.0325427 , -0.03195765,
       -0.04134232, -0.03124091, -0.03002473, -0.04816688, -0.03594143,
       -0.03686319, -0.03846411, -0.03047564, -0.03706354, -0.05015702], #0.6
    [-0.02999721, -0.04370585, -0.03653543, -0.03115156, -0.0304244 ,
       -0.04087976, -0.02867037, -0.02974984, -0.0460611 , -0.03052084,
       -0.03560629, -0.03729879, -0.02316285, -0.03868826, -0.04592632], #0.65
    [-0.02877252, -0.03987509, -0.03604689, -0.02739666, -0.03161315,
       -0.0420476 , -0.02836899, -0.02334352, -0.03969527, -0.02432262,
       -0.0352145 , -0.03652477, -0.02172601, -0.03455551, -0.04937576], #0.7
    [-0.02917136, -0.04390904, -0.03292356, -0.03416868, -0.03670397,
       -0.04446186, -0.02784955, -0.02927159, -0.04347145, -0.02599104,
       -0.03276045, -0.04068267, -0.02246773, -0.03681427, -0.05986066], #0.75
    [-0.03251897, -0.03984398, -0.03564422, -0.02581716, -0.03873215,
       -0.04386529, -0.02834308, -0.02607837, -0.0425306 , -0.016579  ,
       -0.03115067, -0.03458558, -0.02188727, -0.03482287, -0.05255639] #0.8
]


mae = [
    [-0.15891208, -0.17190395, -0.16042931, -0.15327281, -0.14417127,
       -0.16327239, -0.14418458, -0.14482854, -0.19887979, -0.15300159,
       -0.16709067, -0.1670218 , -0.13938338, -0.15230718, -0.16686803], #0.5
    [-0.14883825, -0.1675614 , -0.14748616, -0.14748169, -0.13997827,
       -0.16122939, -0.14607972, -0.13212137, -0.18262712, -0.15313564,
       -0.15337685, -0.16640214, -0.1287241 , -0.15039697, -0.1694681], #0.55
    [-0.14051146, -0.15586159, -0.14167558, -0.12794599, -0.13293532,
       -0.15176309, -0.14077796, -0.13424689, -0.17708856, -0.13717671,
       -0.13828408, -0.15526047, -0.13165183, -0.14313542, -0.16307531], #0.6
    [-0.1372404 , -0.15030398, -0.14066403, -0.12710897, -0.12831851,
       -0.14553835, -0.1343954 , -0.13265506, -0.17584444, -0.12772954,
       -0.13659768, -0.1506461 , -0.11023419, -0.14184335, -0.15915374], #0.65
    [-0.13123459, -0.14323502, -0.13046258, -0.11488045, -0.13495348,
       -0.14496575, -0.12867915, -0.11986048, -0.15634896, -0.11300262,
       -0.13350815, -0.15087491, -0.10894256, -0.13161879, -0.16349307], #0.7
    [-0.13053489, -0.15632278, -0.12939449, -0.1343738 , -0.14443975,
       -0.15691908, -0.12383374, -0.12414367, -0.15763311, -0.11372319,
       -0.13004354, -0.15673111, -0.11658818, -0.14796001, -0.17679061], #0.75
    [-0.13060495, -0.13661993, -0.12456674, -0.10528636, -0.13828535,
       -0.14025938, -0.12061987, -0.11759715, -0.15013779, -0.09608385,
       -0.11330403, -0.13908677, -0.10664832, -0.13157823, -0.15906766] #0.8
]

In [49]:
# Perform one-way ANOVA
r2f_value, r2p_value = f_oneway(*rsquared)
msef_value, msep_value = f_oneway(*mse)
maef_value, maep_value = f_oneway(*mae)


print("One-way ANOVA Results:")
print("r-squared F-value:", r2f_value)
print("r-squared p-value:", r2p_value)

print("Mean square error F-value:", msef_value)
print("Mean square error p-value:", msep_value)

print("Mean absolute error F-value:", maef_value)
print("Mean absolute error p-value:", maep_value)

One-way ANOVA Results:
r-squared F-value: 28.55281256097026
r-squared p-value: 1.6206176757156159e-19
Mean square error F-value: 5.1492173429019825
Mean square error p-value: 0.00012251504642114025
Mean absolute error F-value: 7.1153035856926135
Mean absolute error p-value: 2.5963187827647155e-06


In [50]:
r2_075 = [0.7391932, 0.59699563, 0.77811019, 0.77034099, 0.73011032,
           0.65258079, 0.75767709, 0.78212505, 0.65524694, 0.79146323,
           0.75202244, 0.6927761, 0.83066508, 0.69552579, 0.5458303]

rsquared = [
    [0.45634346, 0.45634309, 0.46720414, 0.53580059, 0.57091602,
       0.46280731, 0.4935618, 0.53677778, 0.382604, 0.56331327,
       0.51347667, 0.51085961, 0.48929966, 0.536598, 0.45951789], #0.5
    [0.56464358, 0.47999782, 0.56571044, 0.57966928, 0.62554679,
       0.51259351, 0.53375816, 0.62961087, 0.50366574, 0.63775923,
       0.56679491, 0.54655718, 0.57584658, 0.58425077, 0.46782443], #0.55
    [0.64646241, 0.53280181, 0.61769064, 0.65956511, 0.67255865,
       0.57570432, 0.60962886, 0.65997764, 0.5450545 , 0.69957268,
       0.65669438, 0.61781818, 0.62690541, 0.64047937, 0.50433829], #0.6
    [0.68777572, 0.56812135, 0.65901243, 0.68438677, 0.71190161,
       0.60903237, 0.67146768, 0.68490953, 0.57545748, 0.75133305,
       0.67938775, 0.64292705, 0.73683909, 0.64411455, 0.57301455], #0.65
    [0.72481275, 0.61634826, 0.68385177, 0.7341573 , 0.72475438,
       0.62767679, 0.70360632, 0.77061093, 0.64673306, 0.80856483,
       0.69638464, 0.66650218, 0.77264233, 0.70101662, 0.57129934], #0.7
    [0.74034652, 0.64788809, 0.72896998, 0.77646792, 0.71746258,
       0.67124173, 0.7561366, 0.78296082, 0.65660295, 0.881005,
       0.75959853, 0.71971786, 0.80844378, 0.73828392, 0.60823481] #0.8
]

# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(r2_075, rsquared[0])
t_statistic2, p_value2 = ttest_ind(r2_075, rsquared[1])
t_statistic3, p_value3 = ttest_ind(r2_075, rsquared[2])
t_statistic4, p_value4 = ttest_ind(r2_075, rsquared[3])
t_statistic5, p_value5 = ttest_ind(r2_075, rsquared[4])
t_statistic6, p_value6 = ttest_ind(r2_075, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_r2075 = pd.DataFrame(
    {
        "R-squared Comparison": ["0.75 vs 0.5", "0.75 vs 0.55", "0.75 vs 0.6", "0.75 vs 0.65", "0.75 vs 0.7", "0.75 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_r2075


,R-squared Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.75 vs 0.5,9.285835,4.813080e-10,2.887848e-09
1,0.75 vs 0.55,6.596402,3.735430e-07,1.120629e-06
2,0.75 vs 0.6,4.050936,3.665098e-04,7.330196e-04
3,0.75 vs 0.65,2.376129,2.457348e-02,3.686022e-02
4,0.75 vs 0.7,0.822342,4.178305e-01,5.013966e-01
5,0.75 vs 0.8,-0.552333,5.851026e-01,5.851026e-01


In [51]:
r2_08 = [0.74034652, 0.64788809, 0.72896998, 0.77646792, 0.71746258,
       0.67124173, 0.7561366 , 0.78296082, 0.65660295, 0.881005  ,
       0.75959853, 0.71971786, 0.80844378, 0.73828392, 0.60823481]

rsquared = [
    [0.45634346, 0.45634309, 0.46720414, 0.53580059, 0.57091602,
       0.46280731, 0.4935618 , 0.53677778, 0.382604  , 0.56331327,
       0.51347667, 0.51085961, 0.48929966, 0.536598  , 0.45951789], #0.5
    [0.56464358, 0.47999782, 0.56571044, 0.57966928, 0.62554679,
       0.51259351, 0.53375816, 0.62961087, 0.50366574, 0.63775923,
       0.56679491, 0.54655718, 0.57584658, 0.58425077, 0.46782443], #0.55
    [0.64646241, 0.53280181, 0.61769064, 0.65956511, 0.67255865,
       0.57570432, 0.60962886, 0.65997764, 0.5450545 , 0.69957268,
       0.65669438, 0.61781818, 0.62690541, 0.64047937, 0.50433829], #0.6
    [0.68777572, 0.56812135, 0.65901243, 0.68438677, 0.71190161,
       0.60903237, 0.67146768, 0.68490953, 0.57545748, 0.75133305,
       0.67938775, 0.64292705, 0.73683909, 0.64411455, 0.57301455], #0.65
    [0.72481275, 0.61634826, 0.68385177, 0.7341573 , 0.72475438,
       0.62767679, 0.70360632, 0.77061093, 0.64673306, 0.80856483,
       0.69638464, 0.66650218, 0.77264233, 0.70101662, 0.57129934], #0.7
    [0.7391932 , 0.59699563, 0.77811019, 0.77034099, 0.73011032,
       0.65258079, 0.75767709, 0.78212505, 0.65524694, 0.79146323,
       0.75202244, 0.6927761 , 0.83066508, 0.69552579, 0.5458303], #0.75
]


# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(r2_08, rsquared[0])
t_statistic2, p_value2 = ttest_ind(r2_08, rsquared[1])
t_statistic3, p_value3 = ttest_ind(r2_08, rsquared[2])
t_statistic4, p_value4 = ttest_ind(r2_08, rsquared[3])
t_statistic5, p_value5 = ttest_ind(r2_08, rsquared[4])
t_statistic6, p_value6 = ttest_ind(r2_08, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_r208 = pd.DataFrame(
    {
        "R-squared Comparison": ["0.8 vs 0.5", "0.8 vs 0.55", "0.8 vs 0.6", "0.8 vs 0.65", "0.8 vs 0.7", "0.8 vs 0.75"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_r208


,R-squared Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.8 vs 0.5,10.811322,1.673435e-11,1.004061e-10
1,0.8 vs 0.55,7.851481,1.493602e-08,4.480807e-08
2,0.8 vs 0.6,5.043606,2.461630e-05,4.923261e-05
3,0.8 vs 0.65,3.216257,3.267775e-03,4.901663e-03
4,0.8 vs 0.7,1.496514,1.457078e-01,1.748494e-01
5,0.8 vs 0.75,0.552333,5.851026e-01,5.851026e-01


In [52]:
r2_07 = [0.72481275, 0.61634826, 0.68385177, 0.7341573 , 0.72475438,
       0.62767679, 0.70360632, 0.77061093, 0.64673306, 0.80856483,
       0.69638464, 0.66650218, 0.77264233, 0.70101662, 0.57129934] #0.7

rsquared = [
    [0.45634346, 0.45634309, 0.46720414, 0.53580059, 0.57091602,
       0.46280731, 0.4935618 , 0.53677778, 0.382604  , 0.56331327,
       0.51347667, 0.51085961, 0.48929966, 0.536598  , 0.45951789], #0.5
    [0.56464358, 0.47999782, 0.56571044, 0.57966928, 0.62554679,
       0.51259351, 0.53375816, 0.62961087, 0.50366574, 0.63775923,
       0.56679491, 0.54655718, 0.57584658, 0.58425077, 0.46782443], #0.55
    [0.64646241, 0.53280181, 0.61769064, 0.65956511, 0.67255865,
       0.57570432, 0.60962886, 0.65997764, 0.5450545 , 0.69957268,
       0.65669438, 0.61781818, 0.62690541, 0.64047937, 0.50433829], #0.6
    [0.68777572, 0.56812135, 0.65901243, 0.68438677, 0.71190161,
       0.60903237, 0.67146768, 0.68490953, 0.57545748, 0.8133305,
       0.67938775, 0.64292705, 0.73683909, 0.64411455, 0.57301455], #0.65
    [0.7391932 , 0.59699563, 0.77811019, 0.77034099, 0.73011032,
       0.65258079, 0.8767709, 0.78212505, 0.65524694, 0.79146323,
       0.8202244, 0.6927761 , 0.83066508, 0.69552579, 0.5458303], #0.75
    [0.74034652, 0.64788809, 0.72896998, 0.77646792, 0.71746258,
       0.67124173, 0.7561366 , 0.78296082, 0.65660295, 0.881005  ,
       0.75959853, 0.71971786, 0.80844378, 0.73828392, 0.60823481] #0.8
]



# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(r2_07, rsquared[0])
t_statistic2, p_value2 = ttest_ind(r2_07, rsquared[1])
t_statistic3, p_value3 = ttest_ind(r2_07, rsquared[2])
t_statistic4, p_value4 = ttest_ind(r2_07, rsquared[3])
t_statistic5, p_value5 = ttest_ind(r2_07, rsquared[4])
t_statistic6, p_value6 = ttest_ind(r2_07, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_r207 = pd.DataFrame(
    {
        "R-squared Comparison": ["0.7 vs 0.5", "0.7 vs 0.55", "0.7 vs 0.6", "0.7 vs 0.65", "0.7 vs 0.75", "0.7 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_r207


,R-squared Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.7 vs 0.5,9.588542,2.413311e-10,1.447987e-09
1,0.7 vs 0.55,6.504176,4.765565e-07,1.429669e-06
2,0.7 vs 0.6,3.604076,1.201368e-03,2.402735e-03
3,0.7 vs 0.65,1.426111,1.648912e-01,1.978694e-01
4,0.7 vs 0.75,-1.180865,2.475902e-01,2.475902e-01
5,0.7 vs 0.8,-1.496514,1.457078e-01,1.978694e-01


In [53]:
mse_075 =   [-0.02917136, -0.04390904, -0.03292356, -0.03416868, -0.03670397,
       -0.04446186, -0.02784955, -0.02927159, -0.04347145, -0.02599104,
       -0.03276045, -0.04068267, -0.02246773, -0.03681427, -0.05986066]

mse = [
    [-0.04244925, -0.05463119, -0.05009597, -0.04321857, -0.03660862,
       -0.04670382, -0.03503022, -0.03695132, -0.06444117, -0.05112117,
       -0.05048046, -0.04733181, -0.03709436, -0.04430145, -0.04983393], #0.5
    [-0.03606417, -0.05172759, -0.04219071, -0.0393957 , -0.03401385,
       -0.04462899, -0.03450002, -0.03089786, -0.05186723, -0.04265159,
       -0.04546227, -0.04447348, -0.03247131, -0.0410435 , -0.05112777], #0.55
    [-0.03140706, -0.04658561, -0.03881302, -0.0325427 , -0.03195765,
       -0.04134232, -0.03124091, -0.03002473, -0.04816688, -0.03594143,
       -0.03686319, -0.03846411, -0.03047564, -0.03706354, -0.05015702], #0.6
    [-0.02999721, -0.04370585, -0.03653543, -0.03115156, -0.0304244 ,
       -0.04087976, -0.02867037, -0.02974984, -0.0460611 , -0.03052084,
       -0.03560629, -0.03729879, -0.02316285, -0.03868826, -0.04592632], #0.65
    [-0.02877252, -0.03987509, -0.03604689, -0.02739666, -0.03161315,
       -0.0420476 , -0.02836899, -0.02334352, -0.03969527, -0.02432262,
       -0.0352145 , -0.03652477, -0.02172601, -0.03455551, -0.04937576], #0.7
    [-0.03251897, -0.03984398, -0.03564422, -0.02581716, -0.03873215,
       -0.04386529, -0.02834308, -0.02607837, -0.0425306 , -0.016579  ,
       -0.03115067, -0.03458558, -0.02188727, -0.03482287, -0.05255639] #0.8
]

# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mse_075, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mse_075, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mse_075, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mse_075, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mse_075, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mse_075, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mse075 = pd.DataFrame(
    {
        "Mean Squared Error Comparison": ["0.75 vs 0.5", "0.75 vs 0.55", "0.75 vs 0.6", "0.75 vs 0.65", "0.75 vs 0.7", "0.75 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mse075


,Mean Squared Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.75 vs 0.5,-40.539743,2.064124e-26,2.476948e-26
1,0.75 vs 0.55,-43.664075,2.663950e-27,7.991850e-27
2,0.75 vs 0.6,-44.849045,1.271609e-27,7.629657e-27
3,0.75 vs 0.65,-40.659543,1.902976e-26,2.476948e-26
4,0.75 vs 0.7,-32.428514,9.425182e-24,9.425182e-24
5,0.75 vs 0.8,-42.904567,4.323836e-27,8.647673e-27


In [54]:
mse_08 = [-0.03251897, -0.03984398, -0.03564422, -0.02581716, -0.03873215,
       -0.04386529, -0.02834308, -0.02607837, -0.0425306 , -0.016579  ,
       -0.03115067, -0.03458558, -0.02188727, -0.03482287, -0.05255639]

mse = [
    [-0.04244925, -0.05463119, -0.05009597, -0.04321857, -0.03660862,
       -0.04670382, -0.03503022, -0.03695132, -0.06444117, -0.05112117,
       -0.05048046, -0.04733181, -0.03709436, -0.04430145, -0.04983393], #0.5
    [-0.03606417, -0.05172759, -0.04219071, -0.0393957 , -0.03401385,
       -0.04462899, -0.03450002, -0.03089786, -0.05186723, -0.04265159,
       -0.04546227, -0.04447348, -0.03247131, -0.0410435 , -0.05112777], #0.55
    [-0.03140706, -0.04658561, -0.03881302, -0.0325427 , -0.03195765,
       -0.04134232, -0.03124091, -0.03002473, -0.04816688, -0.03594143,
       -0.03686319, -0.03846411, -0.03047564, -0.03706354, -0.05015702], #0.6
    [-0.02999721, -0.04370585, -0.03653543, -0.03115156, -0.0304244 ,
       -0.04087976, -0.02867037, -0.02974984, -0.0460611 , -0.03052084,
       -0.03560629, -0.03729879, -0.02316285, -0.03868826, -0.04592632], #0.65
    [-0.02877252, -0.03987509, -0.03604689, -0.02739666, -0.03161315,
       -0.0420476 , -0.02836899, -0.02334352, -0.03969527, -0.02432262,
       -0.0352145 , -0.03652477, -0.02172601, -0.03455551, -0.04937576], #0.7
    [-0.02917136, -0.04390904, -0.03292356, -0.03416868, -0.03670397,
       -0.04446186, -0.02784955, -0.02927159, -0.04347145, -0.02599104,
       -0.03276045, -0.04068267, -0.02246773, -0.03681427, -0.05986066], #0.75
]


# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mse_08, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mse_08, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mse_08, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mse_08, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mse_08, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mse_08, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mse08 = pd.DataFrame(
    {
        "Mean Square Error Comparison": ["0.8 vs 0.5", "0.8 vs 0.55", "0.8 vs 0.6", "0.8 vs 0.65", "0.8 vs 0.7", "0.8 vs 0.75"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mse08


,Mean Square Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.8 vs 0.5,-40.388163,2.288486e-26,2.746183e-26
1,0.8 vs 0.55,-43.519082,2.920152e-27,8.760457e-27
2,0.8 vs 0.6,-44.712544,1.383335e-27,8.300009e-27
3,0.8 vs 0.65,-40.538661,2.065643e-26,2.746183e-26
4,0.8 vs 0.7,-32.335425,1.019455e-23,1.019455e-23
5,0.8 vs 0.75,-42.788841,4.658395e-27,9.316791e-27


In [55]:
mse_07 = [-0.02877252, -0.03987509, -0.03604689, -0.02739666, -0.03161315,
       -0.0420476 , -0.02836899, -0.02334352, -0.03969527, -0.02432262,
       -0.0352145 , -0.03652477, -0.02172601, -0.03455551, -0.04937576]

mse = [
    [-0.04244925, -0.05463119, -0.05009597, -0.04321857, -0.03660862,
       -0.04670382, -0.03503022, -0.03695132, -0.06444117, -0.05112117,
       -0.05048046, -0.04733181, -0.03709436, -0.04430145, -0.04983393], #0.5
    [-0.03606417, -0.05172759, -0.04219071, -0.0393957 , -0.03401385,
       -0.04462899, -0.03450002, -0.03089786, -0.05186723, -0.04265159,
       -0.04546227, -0.04447348, -0.03247131, -0.0410435 , -0.05112777], #0.55
    [-0.03140706, -0.04658561, -0.03881302, -0.0325427 , -0.03195765,
       -0.04134232, -0.03124091, -0.03002473, -0.04816688, -0.03594143,
       -0.03686319, -0.03846411, -0.03047564, -0.03706354, -0.05015702], #0.6
    [-0.02999721, -0.04370585, -0.03653543, -0.03115156, -0.0304244 ,
       -0.04087976, -0.02867037, -0.02974984, -0.0460611 , -0.03052084,
       -0.03560629, -0.03729879, -0.02316285, -0.03868826, -0.04592632], #0.65
    [-0.02877252, -0.03987509, -0.03604689, -0.02739666, -0.03161315,
       -0.0420476 , -0.02836899, -0.02334352, -0.03969527, -0.02432262,
       -0.0352145 , -0.03652477, -0.02172601, -0.03455551, -0.04937576], #0.7
    [-0.02917136, -0.04390904, -0.03292356, -0.03416868, -0.03670397,
       -0.04446186, -0.02784955, -0.02927159, -0.04347145, -0.02599104,
       -0.03276045, -0.04068267, -0.02246773, -0.03681427, -0.05986066], #0.75
       [-0.03251897, -0.03984398, -0.03564422, -0.02581716, -0.03873215,
       -0.04386529, -0.02834308, -0.02607837, -0.0425306 , -0.016579  ,
       -0.03115067, -0.03458558, -0.02188727, -0.03482287, -0.05255639] #0.8
]


# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mse_07, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mse_07, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mse_07, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mse_07, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mse_07, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mse_07, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mse07 = pd.DataFrame(
    {
        "Mean Squared Error Comparison": ["0.7 vs 0.5", "0.7 vs 0.55", "0.7 vs 0.6", "0.7 vs 0.65", "0.7 vs 0.75", "0.7 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mse07

,Mean Squared Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.7 vs 0.5,-40.558327,2.038229e-26,2.445875e-26
1,0.7 vs 0.55,-43.690378,2.620026e-27,7.860078e-27
2,0.7 vs 0.6,-44.864702,1.259405e-27,7.556431e-27
3,0.7 vs 0.65,-40.632205,1.938564e-26,2.445875e-26
4,0.7 vs 0.75,-32.367553,9.921934e-24,9.921934e-24
5,0.7 vs 0.8,-42.879883,4.393048e-27,8.786097e-27


In [56]:
mae_075 = [-0.13053489, -0.15632278, -0.12939449, -0.1343738 , -0.14443975,
       -0.15691908, -0.12383374, -0.12414367, -0.15763311, -0.11372319,
       -0.13004354, -0.15673111, -0.11658818, -0.14796001, -0.17679061]

mae = [
    [-0.15891208, -0.17190395, -0.16042931, -0.15327281, -0.14417127,
       -0.16327239, -0.14418458, -0.14482854, -0.19887979, -0.15300159,
       -0.16709067, -0.1670218 , -0.13938338, -0.15230718, -0.16686803], #0.5
    [-0.14883825, -0.1675614 , -0.14748616, -0.14748169, -0.13997827,
       -0.16122939, -0.14607972, -0.13212137, -0.18262712, -0.15313564,
       -0.15337685, -0.16640214, -0.1287241 , -0.15039697, -0.1694681], #0.55
    [-0.14051146, -0.15586159, -0.14167558, -0.12794599, -0.13293532,
       -0.15176309, -0.14077796, -0.13424689, -0.17708856, -0.13717671,
       -0.13828408, -0.15526047, -0.13165183, -0.14313542, -0.16307531], #0.6
    [-0.1372404 , -0.15030398, -0.14066403, -0.12710897, -0.12831851,
       -0.14553835, -0.1343954 , -0.13265506, -0.17584444, -0.12772954,
       -0.13659768, -0.1506461 , -0.11023419, -0.14184335, -0.15915374], #0.65
    [-0.13123459, -0.14323502, -0.13046258, -0.11488045, -0.13495348,
       -0.14496575, -0.12867915, -0.11986048, -0.15634896, -0.11300262,
       -0.13350815, -0.15087491, -0.10894256, -0.13161879, -0.16349307], #0.7
    [-0.13060495, -0.13661993, -0.12456674, -0.10528636, -0.13828535,
       -0.14025938, -0.12061987, -0.11759715, -0.15013779, -0.09608385,
       -0.11330403, -0.13908677, -0.10664832, -0.13157823, -0.15906766] #0.8
]

# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mae_075, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mae_075, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mae_075, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mae_075, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mae_075, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mae_075, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mae075 = pd.DataFrame(
    {
        "Mean Absolute Error Comparison": ["0.75 vs 0.5", "0.75 vs 0.55", "0.75 vs 0.6", "0.75 vs 0.65", "0.75 vs 0.7", "0.75 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mae075


,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.75 vs 0.5,-46.322229,5.204655e-28,7.806982e-28
1,0.75 vs 0.55,-49.185119,9.901830e-29,2.970549e-28
2,0.75 vs 0.6,-50.096193,5.955961e-29,2.970549e-28
3,0.75 vs 0.65,-45.470889,8.691592e-28,1.042991e-27
4,0.75 vs 0.7,-36.300332,4.301464e-25,4.301464e-25
5,0.75 vs 0.8,-47.514879,2.576446e-28,5.152892e-28


In [68]:
mae_07 = [-0.13123459, -0.14323502, -0.13046258, -0.11488045, -0.13495348,
       -0.14496575, -0.12867915, -0.11986048, -0.15634896, -0.11300262,
       -0.13350815, -0.15087491, -0.10894256, -0.13161879, -0.16349307]

mae = [
    [-0.15891208, -0.17190395, -0.16042931, -0.15327281, -0.14417127,
       -0.16327239, -0.14418458, -0.14482854, -0.19887979, -0.15300159,
       -0.16709067, -0.1670218 , -0.13938338, -0.15230718, -0.16686803], #0.5
    [-0.14883825, -0.1675614 , -0.14748616, -0.14748169, -0.13997827,
       -0.16122939, -0.14607972, -0.13212137, -0.18262712, -0.15313564,
       -0.15337685, -0.16640214, -0.1287241 , -0.15039697, -0.1694681], #0.55
    [-0.14051146, -0.15586159, -0.14167558, -0.12794599, -0.13293532,
       -0.15176309, -0.14077796, -0.13424689, -0.17708856, -0.13717671,
       -0.13828408, -0.15526047, -0.13165183, -0.14313542, -0.16307531], #0.6
    [-0.1372404 , -0.15030398, -0.14066403, -0.12710897, -0.12831851,
       -0.14553835, -0.1343954 , -0.13265506, -0.17584444, -0.12772954,
       -0.13659768, -0.1506461 , -0.11023419, -0.14184335, -0.15915374], #0.65
    [-0.13053489, -0.15632278, -0.12939449, -0.1343738 , -0.14443975,
       -0.15691908, -0.12383374, -0.12414367, -0.15763311, -0.11372319,
       -0.13004354, -0.15673111, -0.11658818, -0.14796001, -0.17679061], #0.75
    [-0.13060495, -0.13661993, -0.12456674, -0.10528636, -0.13828535,
       -0.14025938, -0.12061987, -0.11759715, -0.15013779, -0.09608385,
       -0.11330403, -0.13908677, -0.10664832, -0.13157823, -0.15906766] #0.8
]


# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mae_07, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mae_07, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mae_07, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mae_07, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mae_07, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mae_07, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mae07 = pd.DataFrame(
    {
        "Mean Absolute Error Comparison": ["0.7 vs 0.5", "0.7 vs 0.55", "0.7 vs 0.6", "0.7 vs 0.65", "0.7 vs 0.75", "0.7 vs 0.8"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mae07

,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.7 vs 0.5,-46.530019,4.598760e-28,6.898139e-28
1,0.7 vs 0.55,-49.402481,8.763650e-29,2.629095e-28
2,0.7 vs 0.6,-50.272170,5.404601e-29,2.629095e-28
3,0.7 vs 0.65,-45.508028,8.497661e-28,1.019719e-27
4,0.7 vs 0.75,-36.208484,4.611073e-25,4.611073e-25
5,0.7 vs 0.8,-47.552012,2.521355e-28,5.042710e-28


In [58]:
mae_08 = [-0.13060495, -0.13661993, -0.12456674, -0.10528636, -0.13828535,
       -0.14025938, -0.12061987, -0.11759715, -0.15013779, -0.09608385,
       -0.11330403, -0.13908677, -0.10664832, -0.13157823, -0.15906766] 

mae = [
    [-0.15891208, -0.17190395, -0.16042931, -0.15327281, -0.14417127,
       -0.16327239, -0.14418458, -0.14482854, -0.19887979, -0.15300159,
       -0.16709067, -0.1670218 , -0.13938338, -0.15230718, -0.16686803], #0.5
    [-0.14883825, -0.1675614 , -0.14748616, -0.14748169, -0.13997827,
       -0.16122939, -0.14607972, -0.13212137, -0.18262712, -0.15313564,
       -0.15337685, -0.16640214, -0.1287241 , -0.15039697, -0.1694681], #0.55
    [-0.14051146, -0.15586159, -0.14167558, -0.12794599, -0.13293532,
       -0.15176309, -0.14077796, -0.13424689, -0.17708856, -0.13717671,
       -0.13828408, -0.15526047, -0.13165183, -0.14313542, -0.16307531], #0.6
    [-0.1372404 , -0.15030398, -0.14066403, -0.12710897, -0.12831851,
       -0.14553835, -0.1343954 , -0.13265506, -0.17584444, -0.12772954,
       -0.13659768, -0.1506461 , -0.11023419, -0.14184335, -0.15915374], #0.65
    [-0.13123459, -0.14323502, -0.13046258, -0.11488045, -0.13495348,
       -0.14496575, -0.12867915, -0.11986048, -0.15634896, -0.11300262,
       -0.13350815, -0.15087491, -0.10894256, -0.13161879, -0.16349307], #0.7
    [-0.13053489, -0.15632278, -0.12939449, -0.1343738 , -0.14443975,
       -0.15691908, -0.12383374, -0.12414367, -0.15763311, -0.11372319,
       -0.13004354, -0.15673111, -0.11658818, -0.14796001, -0.17679061], #0.75
]


# Perform t-tests and compute p-values
t_statistic1, p_value1 = ttest_ind(mae_08, rsquared[0])
t_statistic2, p_value2 = ttest_ind(mae_08, rsquared[1])
t_statistic3, p_value3 = ttest_ind(mae_08, rsquared[2])
t_statistic4, p_value4 = ttest_ind(mae_08, rsquared[3])
t_statistic5, p_value5 = ttest_ind(mae_08, rsquared[4])
t_statistic6, p_value6 = ttest_ind(mae_08, rsquared[5])

# Apply Benjamini-Hochberg procedure to obtain adjusted p-values
adjusted_p_values = multipletests([p_value1, p_value2, p_value3, p_value4, p_value5, p_value6], method='fdr_bh')[1]

# Store the results in a pandas DataFrame
df_results_mae08 = pd.DataFrame(
    {
        "Mean Absolute Error Comparison": ["0.8 vs 0.5", "0.8 vs 0.55", "0.8 vs 0.6", "0.8 vs 0.65", "0.8 vs 0.7", "0.8 vs 0.75"],
        "T-Statistic": [t_statistic1, t_statistic2, t_statistic3, t_statistic4, t_statistic5, t_statistic6],
        "P-Value": [p_value1, p_value2, p_value3, p_value4, p_value5, p_value6],
        "Adjusted P-Value": adjusted_p_values
    }
)

# Print the results
df_results_mae08


,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.8 vs 0.5,-45.610116,7.987328e-28,1.198099e-27
1,0.8 vs 0.55,-48.502391,1.458093e-28,4.374280e-28
2,0.8 vs 0.6,-49.446882,8.548311e-29,4.374280e-28
3,0.8 vs 0.65,-44.878967,1.248392e-27,1.498070e-27
4,0.8 vs 0.7,-35.826777,6.166555e-25,6.166555e-25
5,0.8 vs 0.75,-46.946653,3.593913e-28,7.187827e-28


In [69]:
df_results_mae07


,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.7 vs 0.5,-46.530019,4.598760e-28,6.898139e-28
1,0.7 vs 0.55,-49.402481,8.763650e-29,2.629095e-28
2,0.7 vs 0.6,-50.272170,5.404601e-29,2.629095e-28
3,0.7 vs 0.65,-45.508028,8.497661e-28,1.019719e-27
4,0.7 vs 0.75,-36.208484,4.611073e-25,4.611073e-25
5,0.7 vs 0.8,-47.552012,2.521355e-28,5.042710e-28


In [66]:
df_results_mae075

,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.75 vs 0.5,-46.322229,5.204655e-28,7.806982e-28
1,0.75 vs 0.55,-49.185119,9.901830e-29,2.970549e-28
2,0.75 vs 0.6,-50.096193,5.955961e-29,2.970549e-28
3,0.75 vs 0.65,-45.470889,8.691592e-28,1.042991e-27
4,0.75 vs 0.7,-36.300332,4.301464e-25,4.301464e-25
5,0.75 vs 0.8,-47.514879,2.576446e-28,5.152892e-28


In [67]:
df_results_mae08

,Mean Absolute Error Comparison,T-Statistic,P-Value,Adjusted P-Value
0,0.8 vs 0.5,-45.610116,7.987328e-28,1.198099e-27
1,0.8 vs 0.55,-48.502391,1.458093e-28,4.374280e-28
2,0.8 vs 0.6,-49.446882,8.548311e-29,4.374280e-28
3,0.8 vs 0.65,-44.878967,1.248392e-27,1.498070e-27
4,0.8 vs 0.7,-35.826777,6.166555e-25,6.166555e-25
5,0.8 vs 0.75,-46.946653,3.593913e-28,7.187827e-28
